# Predict Neutrino Direction with an LSTM

Using a Tensorflow LSTM layer using the event time steps to the input to predict the Neutrino Direction azimuth and zenith angle

## Imports

In [1]:
# Standard library imports
import os
import random
import math
import logging
from sys import getsizeof
import sys
sys.path.append('..')

# Third-party library imports
import numpy as np
import pandas as pd
import dask.dataframe as dd
import tensorflow as tf


# Typing imports
from typing import List, Tuple

from scripts.utils import seed_it_all, compose_event_df, reduce_mem_usage, convert_bytes_to_gmbkb


2023-02-23 15:06:01.074512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 15:06:01.170277: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-23 15:06:01.170290: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-23 15:06:01.680862: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Variables

In [2]:
# Parameters
IS_TRAINING = True # Whether to train the model
SEED=10

TIME_LIMIT_HOURS = 1

PULSE_AMOUNT = 100 # Amount of pulses to use for features
BATCH_SIZE = 32
FEATURES = [ 'time', 'charge', 'auxiliary', 'x', 'y', 'z'] # Which features to use as the model input

# Directories
DATA_DIR = "../data"
SET = 'train' if IS_TRAINING else 'test'

# logging
LOG_LEVEL = logging.INFO

In [26]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Logging

In [3]:
# Setup logging
logging.basicConfig(filename='artifacts/info.log', level=LOG_LEVEL, format='%(asctime)s %(levelname)s %(message)s')

## Functions

In [4]:
seed_it_all(SEED)
# set the seed for the random number generator
tf.random.set_seed(42)


### For optimization

## Load the dataframes

In [5]:
sensor_dtypes = { 'x': 'float16', 'y': 'float16', 'z': 'float16' }
sensor_geometry_df = pd.read_csv(f'{DATA_DIR}/sensor_geometry.csv', dtype=sensor_dtypes)
sensor_geometry_df.head(1)

,sensor_id,x,y,z
0,0,-256.25,-521.0,496.0


In [6]:
convert_bytes_to_gmbkb(getsizeof(sensor_geometry_df))

'70.69 KB'

In [7]:

meta_dtypes = {'batch_id': 'int16', 'event_id': 'Int64', 'first_pulse_index': 'int32', 'last_pulse_index': 'int32', 'azimuth': 'float16', 'zenith': 'float16'}
meta_df = pd.read_parquet(f'{DATA_DIR}/{SET}_meta.parquet').astype(meta_dtypes)
meta_df.head(1)

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.03125,2.087891


In [8]:
convert_bytes_to_gmbkb(getsizeof(meta_df))

'2.83 GB'

In [9]:
batch_directory = f'{DATA_DIR}/{SET}'
batch_file_paths = [f'{batch_directory}/{file}' for file in os.listdir(batch_directory) if os.path.isfile(os.path.join(batch_directory, file))]
print('First batch file path 3 Samples:')
batch_file_paths[:3]


First batch file path 3 Samples:


['../data/train/batch_540.parquet',
 '../data/train/batch_115.parquet',
 '../data/train/batch_136.parquet']

In [10]:
sample_batch_df= pd.read_parquet(batch_file_paths[1])
convert_bytes_to_gmbkb(getsizeof(meta_df))

'2.83 GB'

## Build the dataset

In [11]:
def get_event_data(batch: pd.DataFrame, event_id: int, sequence_length: int, sensor_geometry: pd.DataFrame, meta_data: pd.DataFrame):
    # The event dataframe with a list of pulse readings
    event_data = batch[batch['event_id'] == event_id]
    
    merged_df = pd.merge(event_data, sensor_geometry, on='sensor_id', how='left')
    
    # get the first N pulses with N being the sequence length
    sequence = merged_df.head(sequence_length)[FEATURES]
    n_missing = 100 - len(sequence)
    if n_missing > 0:
        df_missing = pd.DataFrame(0, index=np.arange(n_missing), columns=sequence.columns)
        sequence = pd.concat([sequence, df_missing])
    
    # get the target labels 
    target_labels = meta_data[meta_data['event_id'] == event_id][['azimuth', 'zenith']].values[0] 
    
    # reshape the sequence and target labels to be fed into the model
    return np.reshape(sequence.values, (sequence_length, len(FEATURES))), np.reshape(target_labels, (1, 2))

In [12]:
import tensorflow as tf

# define a generator function
def data_generator(
    batch_paths:List[str],
    sensor_geometry:pd.DataFrame,
    meta_data: pd.DataFrame, 
    sequence_length:int,
    batch_size:int=BATCH_SIZE
):
    """Emits a single event training example to be called by the model.fit_generator() method.

    Args:
        batch_paths (List[str]): A list of paths to the batch files
        sensor_geometry_df (pd.DataFrame): The sensor geometry dataframe
        meta_df (pd.DataFrame): The dataframe containing the meta data
        sequence_length (int): The length of the pulse sequence to use for training

    Yields:
        _type_: _description_
    """
    batch_dtypes = {'event_id': 'int32', 'sensor_id': 'int16', 'time': 'int32', 'charge': 'float16', 'auxiliary': 'int8'}
    
   
                
    for batch_path in batch_paths:
        
        batch = pd.read_parquet(batch_path).reset_index().astype(batch_dtypes)
        
        output_batch_x = None
        output_batch_y = None
        
        for event_id in batch['event_id'].unique():
            
            x_batch, y_batch = get_event_data(batch, event_id, sequence_length , sensor_geometry, meta_data)
            
            print(len(x_batch),len(x_batch[0]))

            if output_batch_x is None and output_batch_y is None:
                output_batch_x = tf.constant(x_batch)
                output_batch_y = tf.constant(y_batch)
                logging.debug('Output_batch initializing') 
                
            else:
                output_batch_x = tf.concat([output_batch_x, tf.constant(x_batch)], axis=0)
                output_batch_y = tf.concat([output_batch_y, tf.constant(y_batch)], axis=0)
                
                logging.debug('Output_batch extending: %s', len(output_batch_x))
                
                    
            if len(output_batch_x) == batch_size:
                output = output_batch_x[:], output_batch_y[:]
                output_batch_x = None
                output_batch_y = None
                yield output


In [13]:
# create a generator object
data_gen = data_generator(batch_file_paths, sensor_geometry_df, meta_df, sequence_length=PULSE_AMOUNT)

In [14]:
val = next(data_gen)

100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6
100 6


In [15]:
len(val[0][0]), type(val[0][0])

(100, numpy.ndarray)

In [16]:
print('Printing train input shape of 1 batch (1st index)')
len(val), len(val[0]), len(val[0][0])

Printing train input shape of 1 batch (1st index)


(32, 2, 100)

In [17]:
print('Printing label input shape of 1 batch (1st index)')
len(val), (len(val[0]), len(val[0][0]), len(val[0][1]) ),(len(val[1]), len(val[1][0]), len(val[1][1]) ),(len(val[2]), len(val[2][0]), len(val[2][1]) ),


# len(val[2]), len(val[3]),len(val[4]), len(val[5])

Printing label input shape of 1 batch (1st index)


(32, (2, 100, 1), (2, 100, 1), (2, 100, 1))

In [18]:
# type(val)
## first event, train_data_input, first pulse
val[0][0][0]

array([ 6.01900000e+03,  9.75097656e-01,  1.00000000e+00, -5.26500000e+02,
       -1.56015625e+01, -2.63250000e+02])

## Build the model

In [19]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, CuDNNLSTM

In [27]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(32,input_shape=(PULSE_AMOUNT, len(FEATURES))))
model.add(Dense(2, activation='linear')) # set the number of output neurons to 2 and the activation function to linear

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


In [28]:
# model.get_config()

In [29]:
batch = pd.read_parquet('../data/train/batch_1.parquet').reset_index().astype({'event_id': 'int32', 'sensor_id': 'int16', 'time': 'int32', 'charge': 'float16', 'auxiliary': 'int8'})
x_batch, y_batch = get_event_data(batch, 24, PULSE_AMOUNT , sensor_geometry_df, meta_df)


In [38]:
output_batch = np.array([[x_batch, y_batch]])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1, 2) + inhomogeneous part.

In [36]:
output_batch[:,1]

TypeError: list indices must be integers or slices, not tuple

In [30]:
len([x_batch]), len(y_batch)

(1, 1)

In [31]:
for row in x_batch:
    print('row', len(row))

row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6
row 6


In [32]:

# Train the model 
# with input sequence (num_samples, num_timesteps, num_features),
# and the azimuth and zenith angle values for each sequence.

print(len([x_batch]), len([y_batch]))

x_stack = tf.stack([x_batch])
y_stack = tf.stack([y_batch])


print(len(x_stack), len(y_stack))

model.fit(x_stack, y_stack ,steps_per_epoch=50, epochs=10, batch_size=1)


1 1
1 1
Epoch 1/10
50/50 [==============================] - 1s 6ms/step - loss: 14.5328 - accuracy: 1.0000


In [ ]:
# Evaluate the model
loss = model.evaluate(test_sequences, test_values)

## For scoring

In [ ]:
def angular_dist_score(az_true:float, zen_true:float, az_pred:float, zen_pred:float):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    The lower the angle, the more similar the two vectors are meaning the score is better.
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two Cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against numerical instability
    # that might otherwise occur from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))